# Gridworld

Gridworld is an example of a finite Markov Decision Process, first introduced in Chapter 3, Example 3.8 and Figure 3.5.

The code below calculates the state value function of a random policy based on experience (compare *every-visit Monte Carlo method* from Chapter 5).

The gridworld is represented as a 4x4 array.

In [1]:
import numpy as np
import itertools

def grid_walk(numsteps):
    # Four different actions: north, east, south, west
    steps = np.array([[0,-1], [-1,0], [0,1], [1,0]], dtype=int)
    
    # positions[t,:]: Position at time t
    positions = np.zeros((numsteps+1,2),dtype=int)
    # position_count[x,y]: The number of times we visited position (x,y)
    position_count = np.zeros((5,5), dtype=int)
    # rewards[t]: Reward at time t
    rewards = np.zeros((numsteps+1,), dtype=int)
    
    # Starting position
    initial_position = np.random.randint(5,size=(1,2))
    positions[0,:] = initial_position
    position_count[initial_position[0,0], initial_position[0,1]] += 1

    for i in range(numsteps): 
        # Draw a random step
        step = steps[np.random.randint(4),:]
        
        # If we are at special positions A or B, move to A' or B'
        # and give appropriate reward.
        if np.array_equal(positions[i], [0,1]):
            positions[i+1] = np.array([4,1])
            rewards[i+1] = 10
        elif np.array_equal(positions[i], [0,3]):
            positions[i+1] = np.array([2,3])
            rewards[i+1] = 5
        else:
            target = positions[i] + step
            # If target is outside of the world, produce negative reward and
            # stay where you are
            if ((target[0] < 0 or target[0] > 4 or target[1] < 0 or target[1] > 4)):
                positions[i+1] = positions[i]
                rewards[i+1] = -1
            else:
                positions[i+1,:] = target
                rewards[i+1] = 0
                
        position_count[positions[i+1,0], positions[i+1,1]] += 1
    
    return rewards, positions, position_count

In [2]:
# Play the game
numsteps = 10000
rewards, positions, position_count = grid_walk(numsteps)

# gamma: discount rate
gamma = 0.9
# state_values[i,j]: value of state (i,j)
state_values = np.zeros((5,5))

# For each position (i,j) in the gridworld...
for i,j in itertools.product(range(5), range(5)):
    # See when we visited it
    indexes = np.where(np.all(positions[:-1] == [i,j], axis=1))[0]
    
    # Calculate average discounted return
    discounting = np.zeros((numsteps+1,))
    for index in indexes:
        discounting[(index+1)] += 1
        discounting[(index+2):] += np.cumprod(np.repeat(gamma, numsteps + 1 - index - 2))
    state_values[i,j] = np.dot(rewards, discounting) / len(indexes)

state_values, position_count

(array([[ 2.69647781,  8.48136875,  3.9202212 ,  5.56799172,  0.83541671],
        [ 1.52943283,  2.9095513 ,  2.54283748,  2.63804944,  0.64540999],
        [-0.08600168,  0.68325262,  0.84694514,  0.79721515, -0.22778448],
        [-0.8340392 , -0.39077999, -0.33606378, -0.50772009, -1.19792692],
        [-1.72768181, -1.30576507, -1.17340761, -1.40711759, -2.03788446]]),
 array([[110, 114,  93, 131, 109],
        [229, 239, 266, 267, 261],
        [382, 423, 422, 530, 426],
        [563, 609, 549, 541, 536],
        [722, 777, 654, 548, 500]]))